##  Green Infrastructure Designer web site with RHESSys Workflow. 

Using the **GI (Green Infrastructure)** designer web site, users design green infrastructure via GIS maps and web services to create datasets for RHESSys workflows. <br>  

RHESSysWorkflows provides a series of Python tools for performing [RHESSys](https://github.com/RHESSys/RHESSys) data preparation workflows. These tools build on the workflow system defined by [EcohydroLib](https://github.com/selimnairb/EcohydroLib) and [RHESSysWorkflows](https://github.com/selimnairb/RHESSysWorkflows).

<br>
<br>
This cell loads the GI website. Use the **Help tab** for further instructions on generating GI.

In [ ]:
%%HTML
<iframe id="myiframe" width="1000px" height="900px" src="http://gidesigner.renci.org/GI_Designer_01_31_2017_WithIFrame/GI_HS.html"></iframe>

<script type="text/javascript">

var eventMethod = window.addEventListener;
var eventer = window[eventMethod];
var messageEvent = "message";  

window.addEventListener('message',function(e) {
    var key = e.message ? 'message' : 'data';    
    if(key == "data")
    {
        var data = e[key];    
        var res = data.split("="); 
        var new_key = res[0];
        var new_value = res[1];
        console.log(new_key);
        if(new_key.includes("download_url") )
        {
            IPython.notebook.kernel.execute(new_key + '=\'' + new_value + '\'');
            console.log(new_value);
        }
        else
        {
            var json = JSON.parse(new_value);
            var json_string = JSON.stringify(json);     
            console.log(json_string);               
            IPython.notebook.kernel.execute(new_key + '=\'' + json_string + '\'');        
        }
    };
},false);
</script>

&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;&#9866;

## The steps below assume you have created a new soil and landcover GI design.
This cell imports the necessary python libraries for the notebook 

In [ ]:
import os
import logging
from utilities import hydroshare
import shutil
from utilities.rhessys import *
%matplotlib inline

print 'Finished Step'

The **`RHESSysWorkflow`** class is provided as part of the `rhessys` library (imported above) to streamline your interaction with the `RHESSysWorkflows` [core functions](https://github.com/selimnairb/RHESSysWorkflows).
 <br>

Create an instance of the `RHESSysWorkflow` class and assign it to the variable `w`. <br>
This command will create a directory for the given **project name** of `my_workflow` in the JupyterHub's default data directory. **If the directory exists, you will be prompted to use, or overwrite.** <br>
This notebook, a custom gage is used for **gageid**. <br>
A start date of **2008-01-01**, and an end date of **2010-01-01**. 

In [ ]:
w = RHESSysWorkflow(project_name='my_GIworkflow_02_03_17', 
                    gageid='custom',
                    start_date='2008-01-01',
                    end_date='2010-01-01'
                    )

print 'Finished Step'

The RHESSysWorkflow class uses a logging library to document output and errors. We can display these messages in our notebook by attaching to the logger and redirecting the output to stdout. While this step is not necessary, it will provide us with more verbose output.

In [ ]:
# Get root logger for RHESSysWorkflow and set the output to stdout
root_logger = logging.getLogger()
root_logger.addHandler(logging.StreamHandler(sys.stdout))

print 'Finished Step'

## Retrieve data from HydroShare Resource

The first step to retrieve a resource from HydroShare is to create a connection as follows: <br>
**Note** you may be required to enter your credentials.

In [ ]:
hs = hydroshare.hydroshare()

print 'Finished Step'

The UNC Institute for the Environment has prepared a HydroShare Resource for **Dead Run 5 watershed, Baltimore County, Maryland, USA** [with rain gardens](https://www.hydroshare.org/resource/6dbb0dfb8f3a498881e4de428cb1587c/) 
<br>


If you follow the web link, or move your mouse cursor over the above web link, you will note the web link has nas numbers and text after **https://www.hydroshare.org/resource/**. This text string is a **resource-id**. Here the resource id is **c397f68fbc9847d6a717b0a2c2fe7671** and is needed to retrieve the resource from HydroShare. <br>
<br>
**Note:**  <br>
[1] If you use are using this notebook as a template for another **resource-id**, cells below, marked with &#9971; ,indicate where users will need to modify the cell to work with their data.  <br>
[2] If you have used this **resource-id** before in your userspace, you will be prompted to overwrite this data. <br>


In [ ]:
resource_id = '6dbb0dfb8f3a498881e4de428cb1587c'
content = hs.getResourceFromHydroShare(resource_id)

print 'Finished Step'

After the resource has finished downloading, the next step is to unzip the folder in your workspace. <br>
We know the name of the zipped file `ws18.zip` from the above step. <br>

&#9971; If you are using a different resource, you will be required to change the **hs.content['ws18.zip']**, line below. <br>
Here, a new folder called `RHESSys_ETV` is created in the **workspace folder** created by the **RHESSysWorkflow object** to keep the data in a separate location from the following steps. The **zipname** variable, is the name defined in hs.content["MY_FILE"]. 

In [ ]:
#########################################################
## Variables used in cells below
zipfolder = w.sub_project_folder + '/RHESSys_ETV'
zipname = 'DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden'

#########################################################
## Commands
## Create Folder
w.create_path(zipfolder)
## name of zip file and path
zipfilepathname = hs.content[zipname + '.zip']
## Unzip the file
w.unzip_etv_zip_file_at_path(zipfilepathname, zipfolder)

print 'Finished Step'

## Register DEM

The **os.environ["GDAL_DATA"]** indicates to PYTHON where the GDAL installation is. If you have compiled GDAL (i.e. for specialized libraries) in your environment, change this value.
<br/><br/>

**RegisterDEM** will result in the DEM being copied to your project directory, and the DEM spatial resolution will determine the default spatial resolution of your project (i.e. other rasters imported to your project will by default be resampled to the match the DEM resolution and spatial reference). Also, the DEM extent will be used to determine the bounding box for the study area; a polygon of the DEM extent will be generated and saved as a shapefile in your project directory.
<br/><br/>
&#9971; **Note: ** Change the `dem_name` variable below to match the DEM within your resource.


In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder, zipname] defined above 

os.environ["GDAL_DATA"] = "/opt/conda/share/gdal"
dem_name = 'DEM.tif'

demfile = zipfolder + '/' + zipname + '/' + dem_name
options = " -p " + w.sub_project_folder + " -d " + demfile + " -b \"DEM\""

#########################################################
## Commands
w.RegisterDEM(w.sub_project_folder, demfile)

print 'Finished Step'

## Import Gage
We will use the **RegisterGage** tool to tell EcohydroLib the coordinates of the streamflow gage. This tool takes as input a point shapefile containing one or more points; the WGS84 lat-lon coordinates for the desired gage will be extracted from the shapefile. These coordinates will be written to the metadata store, and a new point shapefile containing a point only for the selected gage will be created in the project directory. <br/><br/>
You will need to determine the **shape field** with the **id_attribute**. You can use GIS products from GDAL, ESRI, QGIS to determine these values.
<br/>
**Register streamflow gage** coordinates from a point shapefile into metadata store for a project directory, copying the shapefile into the project directory in the process <br/>
<br/> <br/>
&#9971; **Note: ** Change the `gage_name` variable below to match the gage shapefile within your resource.


In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder, zipname] defined above 
gage_name = 'gage.shp'

gagefile = zipfolder + '/' + zipname + '/' + gage_name
## The name of the layer within the gage shapefile where gage points are located.
layername = 'gage'
## The name of the attribute field that uniquely identifies gage points.
id_attribute = 'gage_id'
## The gage ID that uniquely identifies the gage point.
id_value = '01589312'

#########################################################
## Commands
w.RegisterGage(w.sub_project_folder, gagefile, layername, id_attribute, id_value)

print 'Finished Step'

## Retrieve GI Soil Design

This step retrieves the GI soil dataset from the GI web services. <br>


In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder] defined above 
soil_zipfolder = zipfolder + '/GI_Soils'

#########################################################
## Commands
##print(soil_download_url);
## Note constant from GI web service
zipfilepathname = 'GI_Soil.zip'
input_location = soil_zipfolder + '/' + zipfilepathname

w.create_path(soil_zipfolder)

!wget $soil_download_url -P $soil_zipfolder
w.unzip_etv_zip_file_at_path(input_location, soil_zipfolder)

print 'Finished Step'

By default, the GI web services creates a GI soil shapefile called 'MapunitPolyExtended.shp'.<br>
These files need to be copied to the **project directory**

In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder] defined above 


#########################################################
## Commands
soil_filename = 'MapunitPolyExtended'   ##Constant Name
soil_fullpathname_Src = soil_zipfolder
soil_fullpathname_Dec = w.sub_project_folder

for filename in os.listdir(soil_fullpathname_Src):
    if(filename.startswith(soil_filename)):
        shutil.copy(soil_fullpathname_Src + '/' + filename, soil_fullpathname_Dec)

print 'Finished Step'

The RHESSys workflows automatically adds metadata required by the proceeding functions/tools. If the user has created a new GI Soil data layer, she/he will need to modify the metadata to include soils. <br>
<br>
The **[GI_Soil.py](https://github.com/leonard-psu/JUPYTER-Notebooks/blob/master/GI_Soil.py) ** python script does this for you. The **inputs** are: <br>
[1] **-s** MapunitPolyExtended.shp, the name of the soil shapefile. <br>
[2] **-p** The directory to which metadata, intermediate, and final files should be saved.<br>
[3] **--overwrite** Overwrite existing SSURGO features shapefile in project directory.  If not specified, program will halt if a dataset already exists.<br>

In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder] defined above 
soil_python_location = '/home/jovyan/work/notebooks/GI_Soil.py'

#########################################################
## Commands
soil_shapefile = 'MapunitPolyExtended.shp' ##Constant from web service
python_file_and_inputs = soil_python_location + ' -s ' + soil_shapefile + ' --overwrite -p ' + w.sub_project_folder
print(python_file_and_inputs)
%run $python_file_and_inputs

print 'Finished Step'

## Generate Soil Properties From SSURGO
Get SSURGO features for a bounding box. The following attributes will be rasterized: ['ksat', 'pctClay', 'pctSilt', 'pctSand', 'porosity', 'fieldCap', 'avlWatCap', 'drnWatCont', 'brockdepmin']. Features with null values for a particular attribute will receive a raster value of 0.


In [ ]:
#########################################################
## Commands
output = w.GenerateSoilPropertyRastersFromSSURGO(w.sub_project_folder)
#print(output)

print 'Finished Step'

## Retrieve GI Landcover Design

This step retrieves the GI landcover dataset from the GI web services. <br>


In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder] defined above 
lc_zipfolder = zipfolder + '/GI_Landuse'

#########################################################
## Commands
##print(landuse_download_url);
## Note constant from GI web service
zipfilepathname = 'GI_Landuse.zip'
input_location = lc_zipfolder + '/' + zipfilepathname

w.create_path(lc_zipfolder)

!wget $landuse_download_url -P $lc_zipfolder
w.unzip_etv_zip_file_at_path(input_location, lc_zipfolder)

print 'Finished Step'

In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder] defined above 
lc_zipfolder = zipfolder + '/GI_Landuse'
lc_name = 'GI_LandCover.tif'

raster_type = 'landcover'
raster_sample = 'bilinear'
lc_fullpathname =  lc_zipfolder + '/' + lc_name

options = " -p " + w.sub_project_folder + " -t " + raster_type + " -r " + lc_fullpathname + " --force -s " + raster_sample

#########################################################
## Commands
w.RegisterRasterOptions(options)

print 'Finished Step'

## Register LAI
Here the resource or user supplied **LAI** is registered.
<br/> <br/>
&#9971; **Note: ** Change the `lai_name` variable below to match the LAI raster dataset within your resource. Skip this step, if your resource has no LAI.

In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder, zipname] defined above 
lai_name = 'lai.tif'

lai_file = zipfolder + '/' + zipname + '/' + lai_name

raster_type = 'lai'
raster_sample = 'bilinear'

options = " -p " + w.sub_project_folder + " -t " + raster_type + " -r " + lai_file + " --force -s " + raster_sample

#########################################################
## Commands
w.RegisterRasterOptions(options)

print 'Finished Step'

## Create a new GRASS GIS Location

RHESSys requires that all spatial data used to create a world file and flow table for a RHESSys model be stored in a GRASS GIS mapset. We'll start building these data in RHESSysWorkflows by creating a new GRASS location and mapset within our project directory, and importing our DEM into this mapset using the **CreateGRASSLocationFromDEM** tool. <br/>
<br/> <br/>
&#9971; **Note: ** Change the `description` variable below to match your resource description.

In [ ]:
#########################################################
## Variables
description = '"Dead Run 5 near Catonsville, 3m model"'

#########################################################
## Commands
output = w.CreateGRASSLocationFromDEM(w.sub_project_folder, description)
#print(output)

print 'Finished Step'

## Import RHESSys Code
To create worldfiles and flow tables RHESSysWorkflows needs a copy of RHESSys source code. RHESSysWorkflows also uses the new RHESSys ParamDB database and Python libraries to generate vegetation, soil, land use and other parameters needed by RHESSys. RHESSysWorkflows is only compatible with the pre-release version of RHESSys 5.16 and later versions of the code. At present, and for first-time users, the most reliable way to import ParamDB and RHESSys source code into your project is to download the code from GitHub using the **ImportRHESSysSource** tool. However, this tool is also capable of importing RHESSys source code stored on your computer. This allows you to import the code from a previous RHESSysWorkflows project; ParamDB is always downloaded from GitHub, even when RHESSys source code is imported from a local source.

The `ImportRHESSysSource` function downloads and compiles the latest [RHESSys](https://github.com/RHESSys/RHESSys) source code in the user defined project folder. This make take a couple minutes.   

In [ ]:
#########################################################
## Commands

output = w.ImportRHESSysSource(w.sub_project_folder)
#print(output)

print 'Finished Step'

## Import Climate Data

Because of the great variability of climate data formats, and the complexity of time-series workflows, we have chosen to focus development effort on RHESSysWorkflows toward making it easier to acquire and manipulate geospatial data required for building RHESSys work files and flow tables. This means that the modeler is responsible for building the climate data necessary for building RHESSys world files and performing model runs.

RHESSysWorkflows provides the **ImportClimateData** tool to import RHESSys climate data into your project.


In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder, zipname] defined above 
climate_name = 'rhessys/clim'

climate_file = zipfolder + '/' + zipname + '/' + climate_name

w.climate_data_fullpathname = climate_file;
#########################################################
## Commands
output = w.ImportClimateData(w.sub_project_folder, w.climate_data_fullpathname)
#print(output)

print 'Finished Step'

## Delineate Watershed
RHESSysWorkflows automates the process of delineating your study watershed based on the location of the streamflow gage registered in the workflow using EcohydroLib. As part of this process, many datasets needed by RHESSys will be derived from the DEM. To delineate the watershed we use the **DelineateWatershed** tool.

Here the **cell_size** parameter specifies the minimum size (in DEM cells) for subwatersheds generated by the GRASS command r.watershed.

The **area_estimate** parameter allows you to provide a guess of the area (in sq. km) of the delineated watershed. DelineateWatershed will report whether the watershed is within 20% of the area. You can view the delineated watershed in GRASS by displaying the raster map named basin. If the area or the shape of the delineated watershed differs greatly from what you expect, you may need to vary how DelineateWatershed snaps your streamflow gage onto the stream network. This is accomplished by either changing the -s (a.k.a. --streamThreshold) or stream threshold parameter and/or the -w (a.k.a. --streamWindow) parameter passed to r.findtheriver.
<br/> <br/>
&#9971; **Note: ** Change the `cell_size` and `area_estimate` variables below to match your resource watershed property. Uncomment the `print(output)` statement to evaluate the area estimate.## Step 9 Delineate Watershed

In [ ]:
#########################################################
## Variables

cell_size = 3000
area_estimate = 1.0

#########################################################
## Commands
output = w.DelineateWatershed(w.sub_project_folder, cell_size, area_estimate)
print(output)

print 'Finished Step'

## Generate Patch Map

RHESSysWorkflows provides **GeneratePatchMap**, an automated tool for creating gridded and clumped patch maps. Gridded patch maps consist of a regular grid of the same resolution and extent as the DEM; clumped maps can be created using elevation or topographic wetness index rasters. Modelers can also use custom patch maps registered via EcohydroLib's RegisterRaster tool and imported into GRASS using ImportRasterMapIntoGRASS. <br>
<br>
<br>
**Options** include the following: <br>
**-t {grid,clump}**             Type of patch to be generated: uniform grid or clumps based on elevation. <br>
**-c {elevation,wetness_index}**             Type of patch to be generated: uniform grid or clumps based on elevation.<br>
**-z**             Use patch map as zone map even if zone map is already defined. By default if a zone map is present, this script will not set the patch map as the zone map.<br>
**--overwrite**           Overwrite existing datasets in the GRASS mapset. If not specified, program will halt if a dataset already exists.<br>

In [ ]:
#########################################################
## Variables
grid_options =  " -t grid --overwrite"

patch_options = " -p " + w.sub_project_folder + grid_options

#########################################################
## Commands
output = w.GeneratePatchMapOptions(patch_options)
#print(output)

print 'Finished Step'

## Process Soil Texture Map

Since we used EcohydroLib's SSURGO tools to generate percent sand and percent clay raster maps for our watershed, we can use the GRASS add-on r.soils.texture to generate USDA soil texture classes, for which RHESSys's ParamDB contains parameters. It is also possible to use custom soil maps (refer to the Tutorial).

To generate our soil texture map in GRASS, as well as the corresponding RHESSys soil definition files, use the **GenerateSoilTextureMap** tool as follows: <br>
<br>
<br>
**Options** include the following: <br>
**--defonly**             Only generate soil definition files, do not try to create soil texture map. Map must already exist. <br>
**--overwrite**           Overwrite existing datasets in the GRASS mapset. If not specified, program will halt if a dataset already exists.<br>

In [ ]:
#########################################################
## Variables
soil_options =  " --overwrite"

#########################################################
## Commands
output = w.GenerateSoilTextureMap(w.sub_project_folder, options=soil_options)
#print(output)

print 'Finished Step'

## Register Leaf Coverage
Here the resource or user supplied **Lead Coverage** is registered.
<br/> <br/>

**Register raster** into metadata store for a project directory,
copying the raster into the project directory in the process. <br/> <br/>
**Raster type** must be one of ** landcover,roof_connectivity,soil,lai,patch,zone,isohyet,leafc,rootdepth,roads,stream_burned_dem ** <br/> <br/>
**resampleMethod ** Method to use to resample raster to DEM extent and spatial reference (if necessary). Defaults to 'near'. **{near,bilinear,cubic,cubicspline,lanczos,average,mode}**
<br/> <br/>

&#9971; **Note: ** Change the `leafc_name` variable below to match the LAI raster dataset within your resource. Skip this step, if your resource has no LAI.

In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder, zipname] defined above 
leafc_name = 'leafc.tif'

leafc_file = zipfolder + '/' + zipname + '/' + leafc_name

raster_type = 'leafc'
raster_sample = 'bilinear'

options = " -p " + w.sub_project_folder + " -t " + raster_type + " -r " + leafc_file + " --force -s " + raster_sample

#########################################################
## Commands
w.RegisterRasterOptions(options)

print 'Finished Step'

## Register Root Depth
Here the resource or user supplied **rootdepth** is registered.
<br/>

In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder, zipname] defined above 
rootdepth_name = 'rootdepth.tif'

rootdepth_name_file = zipfolder + '/' + zipname + '/' + rootdepth_name

raster_type = 'rootdepth'
raster_sample = 'bilinear'

options = " -p " + w.sub_project_folder + " -t " + raster_type + " -r " + rootdepth_name_file + " --force -s " + raster_sample

#########################################################
## Commands
w.RegisterRasterOptions(options)

print 'Finished Step'

## Generate derived landcover maps

Now we generating LAI, isohyet, roads and landcover maps from your project directory into GRASS using the **ImportRasterMapIntoGRASS** tool. <br>
**--type** {all,landcover,roof_connectivity,soil,lai,patch,zone,isohyet,leafc,rootdepth,roads,stream_burned_dem}  <br>
**--method** {none,nearest,bilinear,bicubic} <br>
<br/> <br/>
&#9971; **Note: ** Skip any **ImportRasterMapIntoGRASS** steps below if you did not register the rasters above.

In [ ]:
#########################################################
## Variables
raster_type = 'landcover'
raster_sample = 'nearest'

#########################################################
## Commands
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, raster_type, raster_sample)
#print(output)

print 'Finished Step'

In [ ]:
#########################################################
## Variables
raster_type = 'lai'
raster_sample = 'nearest'

#########################################################
## Commands
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, raster_type, raster_sample)
#print(output)

print 'Finished Step'

In [ ]:
#########################################################
## Variables
raster_type = 'leafc'
raster_sample = 'nearest'

#########################################################
## Commands
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, raster_type, raster_sample)
#print(output)

print 'Finished Step'

In [ ]:
#########################################################
## Variables
raster_type = 'rootdepth'
raster_sample = 'nearest'

#########################################################
## Commands
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, raster_type, raster_sample)
#print(output)

print 'Finished Step'

In [ ]:
#########################################################
## Variables
raster_type = 'roof_connectivity'
raster_sample = 'nearest'

#########################################################
## Commands
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, raster_type, raster_sample)
#print(output)

print 'Finished Step'

## Register landcover Rules

The RegisterLandcoverReclassRules function generates landcover raster map reclassification rules ( stratum, landuse, impervious, road, lai-recode ) which are then used by the GenerateLandcoverMaps function to generate derived landscover data for RHESSys.
<br/><br/>
**-k, --generateKnownRules** Generate rules for known landcover type; known types include: ['NLCD2006', 'NLCD2011'] <br/>
**-b, --buildPrototypeRules** Write prototype landcover reclass rules to the project directory. You must edit these rules to match the classes in your own landcover data <br/>
**-r RULEDIR, --ruleDir RULEDIR** The directory where landcover reclass rules can be found; should contain these files ['road.rule', 'impervious.rule','landuse.rule', 'stratum.rule', 'lai-recode.rule'] <br/>
**-l, --includeLaiRules** Make LAI map <br/>
<br/> <br/>
&#9971; **Note: ** Modify **landcover_options** below to match your data inputs.

In [ ]:
#########################################################
## Variables
landcover_options =  " -b --includeLaiRules"

options = " -p " + w.sub_project_folder + landcover_options

#########################################################
## Commands
output = w.RegisterLandcoverReclassRulesOptions(options)
#print(output)

print 'Finished Step'

Once the landcover reclass rules are in place, it is very easy to generate the raster maps derived from the landcover data as well as the vegetation and land use definition files needed by RHESSys using ***GenerateLandcoverMapsOptions*** tool.
<br>
<br>
**Options** include the following: <br>
**-l, --makeLaiMap**      Make LAI map <br>
**--skipRoads**           Do not make roads map <br>
**--defonly**             Only generate landuse and stratum definition files, do not try to create maps. Maps must already exist. <br>
**--overwrite**           Overwrite existing datasets in the GRASS mapset. If not specified, program will halt if a dataset already exists.<br>
<br/> <br/>
&#9971; **Note: ** Modify **landcover_options** below to match your data inputs.


In [ ]:
#########################################################
## Variables
landcover_options =  " --makeLaiMap --overwrite"

options = " -p " + w.sub_project_folder + landcover_options

#########################################################
## Commands
output = w.GenerateLandcoverMapsOptions(options)
#print(output)

print 'Finished Step'

## Generate World Template
Now we are almost ready to create the world file for our watershed. First we must create the template from which the world file will be created. To do this, we'll use the **GenerateWorldTemplate** tool. Fortunately this is very easy because the metadata store contains nearly all the information needed to create the template.

**Options** include the following: <br>
**-c (a.k.a. --climateStation)**      The climate station to associate with the worldfile. Must be one of the climate stations specified in the "climate_stations" key in the "rhessys" section of the metadata<br>
**--aspectMinSlopeOne**           Use slope map with a minimum value of 1.0 to be used for calculating spherical average aspect. Needed for areas of low slope due to limitations of RHESSys grass2world, which truncates slopes <1 to 0.0, which causes spherical average of <br> <br/>
**-v, --verbose **        Print detailed information about what the program is doing<br>




In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder, zipname] defined above 

dir_src = zipfolder + '/' + zipname + '/'  + 'rhessys/defs/'
dir_dst = w.sub_project_folder + '/rhessys/defs/'


#########################################################
## Commands
for filename in os.listdir(dir_src):
    if filename.endswith('.def'):
        shutil.copy( dir_src + '/' + filename, dir_dst)

print 'Finished Step'

<br/> <br/>
&#9971; **Note: ** Modify **world_template_options** below to match your climate data inputs.

In [ ]:
#########################################################
## Variables
world_template_flags =  " -c dr5_composite5"
world_template_options = " -p " + w.sub_project_folder + world_template_flags

#########################################################
## Commands
output = w.GenerateWorldTemplateOptions(world_template_options)
print(output)

print 'Finished Step'

## EDITS REQUIRED using vi at terminal window

cd w.sub_project_folder/rhessys/defs <br>
cp landuse_undeveloped.def landuse_test.def <br>
cp stratum_deciduous_BES.def stratum_test.def <br>
Using a text editor modify the template file. i.e. vi landuse_test.def <br>
vi landuse_test.def Change the number of the first line to match the missing ID above. i.e. 0 Save File and Close <br>
vi stratum_test.def Change the number of the first line to match the missing ID above. i.e. 0 Save File and Close <br>
Change your directory to rhessys\templates <br>
cd w.sub_project_folder/rhessys/templates <br>
Using a text editor modify the template file. i.e. vi template <br>
Modify the number of defs/landuse definitions from 3 to 4. <br>
Modify the number of defs/stratum_ definitions from 3 to 4. <br>
Then add the new def file to the template. <br>


Now use the **CreateWorldfile** tool to create a world file using this template. <br/>
Note: Output from CreateWorldfile has been piped to output variable. For debugging, uncomment print statement (i.e. remove #).

In [ ]:
#########################################################
## Commands
output = w.CreateWorldfile(w.sub_project_folder)
#print(output)

print 'Finished Step'

## Create Flow Table
As with worldfile creation, at this point in the workflow, RHESSysWorkflows's metadata store contains all the information needed to create a flow table using the createflowpaths (CF) RHESSys program. We'll use CreateFlowtable to create our flow table.
<br><br>
This will result in the creation of a flow table called world.flow in the flow directory of your rhessys directory. Now we have almost everything we need to run RHESSys simulations.
<br><br>
See the RHESSysWorkflows tutorial to learn how to [route surface flows for road pixels directly to the stream](https://github.com/selimnairb/RHESSysWorkflows#creating-the-flow-table) and [create surface flow tables using a roof connectivity map](https://github.com/selimnairb/RHESSysWorkflows#creating-a-surface-flow-table-using-a-roof-connectivity-map)
<br>
<br>
**Options** include the following: <br>
**--routeRoads**      Tell createflowpaths to route flow from roads to the nearest stream pixel (requires roads_rast to be defined in metadata)<br>
**--routeRoofs** Tell createflowpaths to route flow from roof tops based on roof top connectivity to nearest impervious surface
(requires roof_connectivity_rast and impervious_rast to be defined in metadata)<br>
**-f, --force ** Run createflowpaths even if DEM x resolution does not match y resolution<br>
**--ignoreBurnedDEM ** Ignore stream burned DEM, if present. Default DEM raster will be used for all operations. If not specified and if stream burned raster is present, stream burned DEM will be used for generating the flow table.<br>
**-v, --verbose **        Print detailed information about what the program is doing<br>


<br/> <br/>
&#9971; **Note: ** Modify **flow_table_options** below to match your data inputs.

In [ ]:
#########################################################
## Variables
flow_table_flags = " --routeRoads -v"
flow_table_options = " -p " + w.sub_project_folder + flow_table_flags

#########################################################
## Commands
output = w.CreateFlowtableOptions(flow_table_options)
print(output)

print 'Finished Step'

## Initializing vegetation carbon and nitrogen stores
RHESSys provides a program called LAIread to initialize vegetation carbon and nitrogen stores in your world file.<br>
<br>
Initializing carbon and nitrogen stores is a multi-step process that involves running LAI read to generate a redefine worldfile, running a 3-day RHESSys simulation to incorporate the redefine worldfile, writing out a new worldfile with initialized vegetation carbon and nitrogen stores. RHESSysWorkflows automates all of these processes for you using the tool RunLAIRead, which can even figure out what date to start the 3-day RHESSys simulation on based on your climate data. <br>
<br>
### Note: The output below is long, however, it is recommended to scroll to the tail end of the output to check for error messages.

If the manual edits failed, users will see error messages here. <br>
If errors occur, **Steps Create World File to Initializing vegetation carbon and nitrogen stores** will need to be repeated.


In [ ]:
#########################################################
## Commands
output = w.RunLAIRead(w.sub_project_folder)
#print(output) ##OUTPUT VERY LONG

print 'Finished Step'

## Creating a RHESSys TEC file
We need one more thing before we can run our model, a TEC file. TEC stands for "temporal event controller". We use a TEC file to tell RHESSys to do things on at certain times during a simulation.

In [ ]:
#########################################################
## Variables
model_start = str(w.model_start_year) + ' ' + str(w.model_start_month) + ' ' + str(w.model_start_day) + ' 1'

tec_file = "tec_daily.txt"
tec_output_location = w.sub_project_folder + "/rhessys/tecfiles/" + tec_file
tec_cmd = ' echo "' + model_start + ' print_daily_on" > '
tec_flags = tec_cmd + tec_output_location
tec_options = " -p " + w.sub_project_folder + tec_flags

#########################################################
## Commands
output = w.RunCmdOptions(tec_options)
print(output)

print 'Finished Step'

## Running RHESSys models
Once you have built a RHESSys model using RHESSysWorkflows, you are free to run your model manually. However this will not capture information about model runs in your project metadata. If you would like to record your runs in your project metadata, use the **RunModel** command.


Because the project metadata knows where RHESSys is installed in your project directory, you don't have to specify the full path of any of the RHESSys input files (e.g. world files, flow tables, TEC files, etc), you only need to specify the filenames. RunModel will echo RHESSys console outlet to the screen (if the -v or verbose option is specified as above), and will always save the same output into a file named 'rhessys.out' stored in the output folder for each model run. The output folder will be named based on the value you provide for the '-pre' or output prefix option. Model output is stored in the output directory of the rhessys directory of your project directory.

For more details on using RunModel see the RHESSysWorkflows [tutorial](https://github.com/selimnairb/RHESSysWorkflows#running-rhessys-models).

<br>
<br>
**Options** include the following: <br>
**-d DESCRIPTION, --description** Description of the model run<br>
**-pre OUTPUTPREFIX** Filename prefix to use for output files, relative to output directory in the RHESSys directory of the project<br>
**-st STARTDATE STARTDATE STARTDATE STARTDATE** Start date and time of the model run, of the form "YYYY M D H"<br>
**-ed ENDDATE ENDDATE ENDDATE ENDDATE** Date date and time of the model run, of the form "YYYY M D H".<br>
**-w WORLDFILE** Filename of the tecfile to use for the model run, specified relative to the tec directory in the RHESSys directory of the projec<br>
**-t TECFILE **        Filename of the tecfile to use for the model run, specified relative to the tec directory in the RHESSys directory of the project<br>
**-r [FLOWTABLES [FLOWTABLES ...]]**Filename(s) of the flow table(s) to use for the model run, specified relative to the flowtable directory in the RHESSys directory of the project. If one flow table is supplied, it will be used for subsurface and surface routing. If two flow tables are supplied the first will be use for subsurface routing, the second for surface<br>
**--basin ** Tell RHESSys to output at the basin spatial level<br>
**--hillslope ** Tell RHESSys to output at the hillslope spatial level<br>
**--zone ** Tell RHESSys to output at the zone spatial level<br>
**--patch ** Tell RHESSys to output at the patch spatial level<br>
**--canopy ** Tell RHESSys to output at the canopy stratum spatial level<br>
**-v, --verbose **        Print detailed information about what the program is doing<br>


<br/> <br/>
&#9971; **Note: ** Modify **options** below to match your data inputs.


In [ ]:
#########################################################
## Variables
## Note dependency of variables [w.model_start_year, w.model_start_month, w.model_start_day] defined above 
## Note dependency of variables [w.model_end_year, w.model_end_month, w.model_end_day] defined above 
## Note dependency of variables [tec_file] defined above 

model_start = str(w.model_start_year) + ' ' + str(w.model_start_month) + ' ' + str(w.model_start_day) + ' 1'
model_end = str(w.model_end_year) + ' ' + str(w.model_end_month) + ' ' + str(w.model_end_day) + ' 1'

description = '"Test model run"'  ##Must use double quotes here
spatial_level = "--basin"
prefix = "test"
world_file = "world"
#tec_file = "tec_daily.txt" ##Defined above step
flow_table = "world.flow"
s_flag = '0.07041256017 133.552915269 1.81282283058'
sv_flag = '4.12459677088 78.3440566535'
gw_flag = '0.00736592779294 0.340346799457'
flow_table_flag = ' -r ' + flow_table + ' -- -s ' + s_flag + ' -sv ' + sv_flag + ' -gw ' + gw_flag

runmodel_flags = '  -d ' + description + ' ' + spatial_level + ' -pre ' + prefix +' -st ' + model_start + ' -ed ' + model_end + ' -w ' + world_file + ' -t ' + tec_file + flow_table_flag
runmodel_options = " -v -p " + w.sub_project_folder + runmodel_flags
##print(runmodel_options)

#########################################################
## Commands
output = w.RunModelOptions(runmodel_options)
print(output)

print 'Finished Step'

## Help with error messages

If you see an error message, such as **returned non-zero exit status 1** check the log file located in the **w.sub_project_folder** folder. The log file name is the same name as the project folder, with **.log** appended.


## Resources

* RHESSys
  * [Setup](http://fiesta.bren.ucsb.edu/~rhessys/setup/setup.html)
  * [Wiki](http://fiesta.bren.ucsb.edu/~rhessys/)
* Data
  * [HydroShare](https://www.hydroshare.org/)
  * [USGS Data and Tools](https://www.usgs.gov/products/data-and-tools/data-and-tools-topics)
  * [USDA Data gateway](https://gdg.sc.egov.usda.gov/)
  * [HydroTerre](http://hydroterre.psu.edu/)
  

## Funding
This work was supported by the following NSF grants

    Award no. 1239678 EAGER: Collaborative Research: Interoperability Testbed-Assessing a Layered Architecture for Integration of Existing Capabilities

    Award no. 0940841 DataNet Federation Consortium.

    Award no. 1148090 Collaborative Research: SI2-SSI: An Interactive Software Infrastructure for Sustaining Collaborative Innovation in the Hydrologic Sciences


## Credits

**Lawrence E. Band** lband@email.unc.edu <br/>
**Brian Miles** brian_miles@unc.edu <br/> 
**Laurence Lin** laurence.lin@icloud.com <br/>
**John Duncan** jon.m.duncan@gmail.com <br/>
**Lorne Leonard** lnl3@psu.edu <br/>
